In [144]:
import requests
import os
import sqlite3
import time
import pandas as pd
from riotwatcher import LolWatcher

In [152]:
conn = sqlite3.connect('C:\sqlite\lol.db') 
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS match_data (
        tier TEXT,
        rank TEXT,
        summonerId TEXT,
        summonerName TEXT,
        leaguePoints INTEGER,
        wins INTEGER,
        losses INTEGER,
        veteran TEXT,
        inactive TEXT,
        freshBlood TEXT,
        hotStreak TEXT,
        puuid TEXT,
        PRIMARY KEY (summonerId, puuid)
    )
''')


In [153]:
api_key = ''
lol_watcher = LolWatcher(api_key)
queue = 'RANKED_SOLO_5x5'
regions = ['na1', 'euw1', 'eun1', 'kr', 'br1', 'tr1', 'la1', 'la2', 'oc1', 'jp1']

In [154]:
top_players_csv = 'top_players.csv'

df_topplayers_all_regions = []

def get_top_players_ranked5x5(region, queue, api_key, cursor):
    assert queue in ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']

    total_users_to_insert = list()

    # Endpoints for challenger, grandmaster, and master tier players
    request_urls = [
        f'https://{region}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{queue}',
        f'https://{region}.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{queue}',
        f'https://{region}.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{queue}'
    ]

    # HTTP header to make requests to Riot API
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:96.0) Gecko/20100101 Firefox/96.0",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "utf-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }

    for x in request_urls:
        response = requests.get(x, headers=headers)
        if response.status_code == 200:
            try:
                print('Name: {} | Region: {} | Tier: {} | Queue: {} | Total Players: {}'.format(
                    response.json()['name'], region, response.json()['tier'], response.json()['queue'], len(response.json()['entries'])))
            except KeyError as e:
                pass
            for y in response.json()['entries']:
                try:
                    y['tier'] = response.json()['tier']
                    y['request_region'] = region
                    y['queue'] = queue

                    # API rate limits
                    time.sleep(1.2)

                    summoner_encrypted = lol_watcher.summoner.by_id(region, y['summonerId'])
                    y['puuid'] = summoner_encrypted['puuid']
                    total_users_to_insert.append(y)
                except KeyError as e:
                    pass

        else:
            print('Request error (@elite). HTTP code {}: {}'.format(response.status_code, response.json()))
            continue

    df_topplayers = pd.DataFrame(total_users_to_insert, columns=['tier', 'rank', 'summonerId', 
    'summonerName', 'leaguePoints', 'wins', 'losses', 'veteran', 'inactive', 'freshBlood', 'hotStreak', 'puuid'])

    df_topplayers['region'] = region

    print('Total users obtained in region {} and queue {}: {}'.format(region, queue, len(total_users_to_insert)))
    print(df_topplayers.head(5))

    df_topplayers_all_regions.append(df_topplayers)

    df_topplayers.to_csv(f'topplayers_{region}.csv', index = False)
    print(f'CSV file saved for {region} region')

    df_topplayers.to_sql('match_data', conn, if_exists = 'replace', index = False)

    conn.commit()

    
    


In [155]:
# Iterate through regions and fetch match data
for region in regions:
    get_top_players_ranked5x5(region, queue, api_key, cursor)

# Concatenate DataFrames from all regions
df_topplayers_all_regions_combined = pd.concat(df_topplayers_all_regions, ignore_index=True)

# Save the overall DataFrame to the main CSV file
df_topplayers_all_regions_combined.to_csv('topplayers.csv', index=False)
print(f"Main CSV file saved to: {os.path.abspath('topplayers.csv')}")

# Close the database connection
conn.close()

Name: LeBlanc's Hunters | Region: na1 | Tier: CHALLENGER | Queue: RANKED_SOLO_5x5 | Total Players: 5
Name: Amumu's Commandos | Region: na1 | Tier: GRANDMASTER | Queue: RANKED_SOLO_5x5 | Total Players: 89
Name: Twisted Fate's Shadows | Region: na1 | Tier: MASTER | Queue: RANKED_SOLO_5x5 | Total Players: 995
Total users obtained in region na1 and queue RANKED_SOLO_5x5: 1089
         tier rank                                         summonerId  \
0  CHALLENGER    I   mhLLqroOptAU2ZU67hzBcJWOgp8_UCiXwl7-s7fjAElD1WZe   
1  CHALLENGER    I    QHRI5njQodt5aesgDHQl4R_kIfI1VeQtMdNnBCRsdhA_kpg   
2  CHALLENGER    I    KPdFHgBkSBJ7XbzunN24XporbIKWt4dJYEFBiQGNnoQgxiU   
3  CHALLENGER    I    ewCfTMmgdXdwlBx4kl2KTVf-O-pAsVNA338m_kY2shXk9S4   
4  CHALLENGER    I  agKQE14_19PZ1T-JvEKw0nhCJ22wLleweql3UKu194SBMd...   

    summonerName  leaguePoints  wins  losses  veteran  inactive  freshBlood  \
0       WanBieJi           501    71      41    False     False        True   
1         Spica9           5